In [8]:
import pandas as pd
import numpy as np
from datetime import datetime

In [9]:
# Load the dataset
df = pd.read_csv("train.csv")

In [10]:
# a) Handle missing values
print("Missing values before handling:\n", df.isnull().sum())

# Impute numeric columns with median and non-numeric with mode
for column in df.columns:
    if df[column].isnull().sum() > 0:
        if df[column].dtype in ['float64', 'int64']:
            df[column] = df[column].fillna(df[column].median())
        else:
            df[column] = df[column].fillna(df[column].mode()[0])

print("\nMissing values after handling:\n", df.isnull().sum())


Missing values before handling:
 Unnamed: 0              0
Name                    0
Location                0
Year                    0
Kilometers_Driven       0
Fuel_Type               0
Transmission            0
Owner_Type              0
Mileage                 2
Engine                 36
Power                  36
Seats                  38
New_Price            5032
Price                   0
dtype: int64

Missing values after handling:
 Unnamed: 0           0
Name                 0
Location             0
Year                 0
Kilometers_Driven    0
Fuel_Type            0
Transmission         0
Owner_Type           0
Mileage              0
Engine               0
Power                0
Seats                0
New_Price            0
Price                0
dtype: int64


## Task (a) Justification – Handling Missing Values

In this Task, the goal was to identify and appropriately handle missing values in the dataset to maintain data quality and enable accurate analysis. After examining the dataset, several columns were found to contain missing entries.

To handle these:

- **Numerical columns** (like `Mileage`, `Engine`, `Power`, etc.) were imputed using the **median** of the respective column. The median is chosen over the mean to avoid distortion caused by extreme outliers, which are common in real-world car data (e.g., luxury cars with unusually high values).
  
- **Categorical columns** (like `Owner_Type`, `Fuel_Type`, `Transmission`, etc.) were imputed using the **mode**, which is the most frequent value. This ensures the imputed value is consistent with the most common category and preserves the distribution of categories in the dataset.

- **No columns were dropped** to retain the maximum amount of data available, which is especially important in datasets where every record might represent a valuable car listing.

This approach ensures the dataset remains representative of the original data while preventing issues that could arise from missing values during modeling or analysis.


In [11]:
# b) Remove units from some columns
def clean_numeric(col, pattern):
    return df[col].astype(str).str.replace(pattern, '', regex=True).str.strip()

df["Mileage"] = clean_numeric("Mileage", r"[^\d.]").astype(float)
df["Engine"] = clean_numeric("Engine", r"[^\d.]").astype(float)
df["Power"] = clean_numeric("Power", r"[^\d.]").astype(float)
df["New_Price"] = clean_numeric("New_Price", r"[^\d.]").astype(float)


In [12]:

# c) One-hot encode Fuel_Type and Transmission
df = pd.get_dummies(df, columns=["Fuel_Type", "Transmission"], drop_first=True)


In [13]:
# d) Create new feature: Car_Age
current_year = datetime.now().year
df["Car_Age"] = current_year - df["Year"]



# Print the DataFrame after point d
print("\n DataFrame after point (d) — with cleaned columns and new Car_Age:\n")
df.head() # You can change 10 to more/less if needed


 DataFrame after point (d) — with cleaned columns and new Car_Age:



,Unnamed: 0,Name,Location,Year,Kilometers_Driven,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price,Fuel_Type_Electric,Fuel_Type_Petrol,Transmission_Manual,Car_Age
0,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,First,19.67,1582.0,126.20,5.0,4.78,12.50,False,False,True,10
1,2,Honda Jazz V,Chennai,2011,46000,First,13.00,1199.0,88.70,5.0,8.61,4.50,False,True,True,14
2,3,Maruti Ertiga VDI,Chennai,2012,87000,First,20.77,1248.0,88.76,7.0,4.78,6.00,False,False,True,13
3,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Second,15.20,1968.0,140.80,5.0,4.78,17.74,False,False,False,12
4,6,Nissan Micra Diesel XV,Jaipur,2013,86999,First,23.08,1461.0,63.10,5.0,4.78,3.50,False,False,True,12


In [14]:

# e) Data operations

# Rename 'Price' before using it in other steps
df.rename(columns={"Price": "Selling_Price"}, inplace=True)

# Select specific columns
selected_df = df[["Name", "Location", "Year", "Mileage", "Engine", "Selling_Price"]]

# Filter cars priced more than 10 lakhs
filtered_df = df[df["Selling_Price"] > 10]

# Mutate: create Mileage_per_CC
df["Mileage_per_CC"] = df["Mileage"] / df["Engine"]

# Arrange: sort by Selling_Price
arranged_df = df.sort_values(by="Selling_Price", ascending=False)

# Summarize: mean, max, min price by Location
summary = df.groupby("Location")["Selling_Price"].agg(["mean", "max", "min"]).reset_index()

# Display output samples
print("\n Selected columns:\n", selected_df.head())
print("\n Filtered cars (Selling_Price > 10 lakhs):\n", filtered_df[["Name", "Selling_Price"]].head())
print("\n New feature - Mileage_per_CC:\n", df[["Name", "Mileage_per_CC"]].head())
print("\n Sorted by Selling_Price:\n", arranged_df[["Name", "Selling_Price"]].head())
print("\n Price summary by Location:\n", summary)



 Selected columns:
                                Name    Location  Year  Mileage  Engine  \
0  Hyundai Creta 1.6 CRDi SX Option        Pune  2015    19.67  1582.0   
1                      Honda Jazz V     Chennai  2011    13.00  1199.0   
2                 Maruti Ertiga VDI     Chennai  2012    20.77  1248.0   
3   Audi A4 New 2.0 TDI Multitronic  Coimbatore  2013    15.20  1968.0   
4            Nissan Micra Diesel XV      Jaipur  2013    23.08  1461.0   

   Selling_Price  
0          12.50  
1           4.50  
2           6.00  
3          17.74  
4           3.50  

 Filtered cars (Selling_Price > 10 lakhs):
                                  Name  Selling_Price
0    Hyundai Creta 1.6 CRDi SX Option          12.50
3     Audi A4 New 2.0 TDI Multitronic          17.74
5   Toyota Innova Crysta 2.8 GX AT 8S          17.50
11   Land Rover Range Rover 2.2L Pure          27.00
12     Land Rover Freelander 2 TD4 SE          17.50

 New feature - Mileage_per_CC:
                         